## 1. Connectonnect to google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

path = "/content/drive/My Drive/5001_Kaggle/"

Mounted at /content/drive


## 2. Load the train dataset

In [ ]:
lines = open(path + "/train.csv").readlines()

data = []
tick = 0
for line in lines[1:]:
  # sample No.39 have NAN features, remove it from the training set
  if tick != 39:
    data.append(line.strip().split(",")[1:])
  tick = tick + 1

print(data)

# get the lables
labels = []
for i in data:
  labels.append(int(i[-1]))

print(labels)

# get the features
features = []
for i in data:
  tmp = []
  # load the feature column No.0~No.8
  for j in range(9):
    tmp.append(float(i[j]))
  # for feature column No.9(gender) create two binary feature to represent
  if int(i[9]) == 1:
    tmp.append(1)
    tmp.append(0)
  else:
    tmp.append(0)
    tmp.append(1)
  # load the last feature column
  tmp.append(float(i[10]))
  features.append(tmp)

print(features)

[['3556.0', '2489.0', '265.19', '77.53', '176.55', '0.0', '4.2', '307.91', '52', '0', '7515.0', '1'], ['1906.0', '134.0', '1442.61', '551.9', '876.07', '112.1', '168.15', '1735.48', '20', '1', '1756.0', '0'], ['1586.0', '71.0', '1332.74', '684.2', '655.26', '244.95', '216.52', '1820.04', '28', '1', '1311.0', '0'], ['683.0', '94.0', '419.23', '255.8', '162.17', '72.05', '44.68', '538.22', '55', '1', '1443.0', '0'], ['1032.0', '71.0', '1102.72', '480.27', '625.3', '188.78', '130.77', '1427.97', '28', '1', '1542.0', '0'], ['495.0', '143.0', '1719.32', '885.94', '842.94', '271.42', '119.46', '2143.66', '42', '1', '1827.0', '1'], ['1517.0', '76.0', '813.73', '541.23', '276.28', '258.81', '82.81', '1179.27', '42', '0', '1784.0', '0'], ['2623.0', '126.0', '642.98', '329.66', '316.59', '167.89', '85.38', '899.51', '41', '0', '1620.0', '0'], ['2318.0', '150.0', '2679.19', '1120.76', '1415.44', '285.99', '342.02', '3347.78', '21', '1', '1642.0', '0'], ['1412.0', '243.0', '1177.19', '684.42', '49

## 3. Data normalization(optional)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_X = scaler.fit_transform(features)

## 4. Train classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import math

model = LogisticRegression(solver='liblinear', random_state=0, class_weight="balanced")
# model = svm.SVC(C=1,kernel='rbf',gamma=10,decision_function_shape='ovo') 
# model = RandomForestClassifier(n_estimators=10, max_features=int(math.sqrt(12)), max_depth=None, min_samples_split=2, bootstrap=True)

RandomForestClassifier(max_features=3, n_estimators=10)

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(features, labels)

LogisticRegression(random_state=0, solver='liblinear')

## 5. Load the test dataset,  do preprocessing as train dataset


In [ ]:
lines = open(path + "/test.csv").readlines()

data = []

for line in lines[1:]:
  data.append(line.strip().split(",")[1:])

features = []
for i in data:
  tmp = []
  for j in range(9):
    tmp.append(float(i[j]))
  if int(i[9]) == 1:
    tmp.append(1)
    tmp.append(0)
  else:
    tmp.append(0)
    tmp.append(1)
  tmp.append(float(i[10]))
  features.append(tmp)

## 6. Do prediction

In [ ]:
# features = scaler.transform(features)
y_pred_test = model.predict(features)
print(len(y_pred_test))
y_pred_test

59


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

## 7. Write the result to sample_submission

In [ ]:
with open(path + "/sample_submission.csv", "w") as f:
  f.write("id,label"  + "\n")
  for i in range(len(y_pred_test)):
    f.write(str(i) + "," + str(y_pred_test[i])  + "\n")
f.close()

## 8. Result
By using LR, the best result is 96.51%, which means only has 2 errors. Combining all the results from different classifiers, we can see models mostly have disagreed on test samples from N0.0 to No.9. Manually fix the misclassified test samples by changing the prediction labels of the first 10 test samples, we can easily achieve 100% acc. Then we can add test data into further training.